# Compare performances

## order-bound

In [ ]:
# load order bound matches with all dictionaries

## order-free

In [ ]:
# load order free matches with all dictionaries

## actual order-free

In [ ]:
# load (actual) order free matches with all dictionaries